## Merge

In [ ]:
import ROOT
import os

def get_root_files(directory):
    
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.root') and not f.startswith('merge')]

def get_unique_filename(directory, base_name):
    counter = 0
    while True:
        if counter == 0:
            filename = os.path.join(directory, f"{base_name}0.root")
        else:
            filename = os.path.join(directory, f"{base_name}{counter}.root")
        
        if not os.path.exists(filename):
            return filename
        counter = counter + 1

def merge_files(file_list, output_file):
    
    merger = ROOT.TFileMerger()
    merger.OutputFile(output_file)
    
    for i in file_list:
        merger.AddFile(i)
    
    merger.Merge()

directory = 'build/root'
files = get_root_files(directory)
output_filename = get_unique_filename(directory, 'merged')

merge_files(files, output_filename)

## Draw graph

In [13]:
import ROOT
import os

def plot_xy_scatter_from_root(input_file, tree_name, x_branch, y_branch, output_image):
    
    file = ROOT.TFile(input_file, "READ")
    tree = file.Get(tree_name)
    
    canvas = ROOT.TCanvas("canvas", "X vs Y Scatter Plot", 800, 600)
    graph = ROOT.TGraph()
    
    points = []
    for event in tree:
        x = getattr(event, x_branch)
        y = getattr(event, y_branch)
        points.append((x, y))

    # Sort points based on x-values
    points.sort(key=lambda p: p[0])

    # Create a TGraph and add sorted points
    graph = ROOT.TGraph()
    for i, (x, y) in enumerate(points):
        graph.SetPoint(i, x, y)

    graph.SetTitle("X vs Y Scatter Plot")
    graph.GetXaxis().SetTitle(x_branch)
    graph.GetYaxis().SetTitle(y_branch)
    graph.SetMarkerStyle(20)  # Circular marker
    graph.SetMarkerSize(0.5)
    graph.SetLineColor(ROOT.kBlue)
    graph.SetLineWidth(2)

    graph.Draw("ALP")  # "A" for axes, "P" for points, "L" for line
    
    canvas.SaveAs(output_image)
    file.Close()

input_file = os.path.join(directory, "merged0.root")
tree_name = "Transportation"
x_branch = "Energy_keV"
y_branch = "Mass_Attenuation"
output_image = "xy_plot.png"

plot_xy_scatter_from_root(input_file, tree_name, x_branch, y_branch, output_image)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file xy_plot.png has been created
